# Risk Premia Strategy Backtests

## Purpose

Write file of 3-asset RP strategy returns, scaled to annualised volatility of 10%. (Rebalanced back to target vol every volLookback days)

Set the end date below. Other parameters are:
- initEq = 10000
- perShareComm = 0.005
- minCommPerOrder = 1
- rebalFrequency = 1 (rebalance frequency in months)
- capFrequency = 1 (frequency to capitalise profits. 0 is "don't capitalise")
- assetVolTarget = 0.05
- volLookback = 60

In [ ]:
backtest_end_date = "2020-11-04"

## Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyreadr
from pathlib import Path
import pickle

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Load prices from RData file
data_path = Path('..') / 'data' / 'prices.RData'
result = pyreadr.read_r(str(data_path))
prices = result['us_etf_prices']

# Convert date and sort
prices['date'] = pd.to_datetime(prices['date'])
prices = prices.sort_values('date').reset_index(drop=True)

print(f"Loaded {len(prices)} rows of price data")
prices.head()

## Plot Cumulative Returns to Verify Data

In [ ]:
# Calculate returns from adjusted close
plot_data = prices.copy()
plot_data = plot_data.sort_values(['ticker', 'date'])
plot_data['returns'] = plot_data.groupby('ticker')['closeadjusted'].pct_change()
plot_data = plot_data.dropna(subset=['returns'])
plot_data['cumreturns'] = plot_data.groupby('ticker')['returns'].apply(
    lambda x: (1 + x).cumprod()
).reset_index(level=0, drop=True)

# Plot
plt.figure(figsize=(12, 6))
for ticker in plot_data['ticker'].unique():
    ticker_data = plot_data[plot_data['ticker'] == ticker]
    plt.plot(ticker_data['date'], ticker_data['cumreturns'], label=ticker)

plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.title('Cumulative Returns (Adjusted Prices)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Backtest Setup

Calculate monthly prices and start/end dates

In [ ]:
# Get month-end dates
prices_with_month = prices.copy()
prices_with_month['year'] = prices_with_month['date'].dt.year
prices_with_month['month'] = prices_with_month['date'].dt.month

monthends = prices_with_month.groupby(['year', 'month'])['date'].max().reset_index()

# Get monthly prices (use closeadjusted)
monthlyprices = prices.merge(monthends, on='date')[['ticker', 'date', 'closeadjusted']]
monthlyprices = monthlyprices.rename(columns={'closeadjusted': 'close'})

# Set dates
startDate = monthlyprices['date'].min()
endDate = pd.to_datetime(backtest_end_date)
initDate = startDate - pd.Timedelta(days=1)

print(f"Start Date: {startDate}")
print(f"End Date: {endDate}")
print(f"Number of monthly periods: {len(monthlyprices['date'].unique())}")
monthlyprices.head()

# Tab 1: Equal Initial Weight Buy and Hold

Here are the inputs for this strategy:

In [ ]:
initEq = 10000  # Initial equity ($1000 to $1 million)
perShareComm = 0.005  # Per-share commission (0.001 to 0.02)
minCommPerOrder = 1  # Minimum commission per order (0 to 5)

This strategy assumes rebalFrequency = capFrequency = 0 (no rebalancing)

In [ ]:
# Calculate initial shares - equal dollar weight
initial_prices = monthlyprices[monthlyprices['date'] == startDate].copy()
shares = initial_prices.copy()
shares['shares'] = np.floor((initEq / 3) / shares['close']).astype(int)
shares = shares[['ticker', 'shares']]

# Merge shares with all prices
ew_norebal = monthlyprices.merge(shares, on='ticker')
ew_norebal['exposure'] = ew_norebal['shares'] * ew_norebal['close']

# Calculate trades
ew_norebal = ew_norebal.sort_values(['ticker', 'date'])
ew_norebal['trades'] = ew_norebal.groupby('ticker')['shares'].diff().fillna(ew_norebal['shares'])
ew_norebal['tradevalue'] = ew_norebal['trades'] * ew_norebal['close']

# Calculate commissions
ew_norebal['commission'] = np.abs(ew_norebal['trades']) * perShareComm
ew_norebal.loc[ew_norebal['commission'] < minCommPerOrder, 'commission'] = minCommPerOrder
ew_norebal.loc[ew_norebal['trades'] == 0, 'commission'] = 0

# Calculate initial cash balance
initial_investment = ew_norebal[ew_norebal['date'] == startDate]
initcashbal = initEq - initial_investment['exposure'].sum() - initial_investment['commission'].sum()

# Create cash rows
cash_rows = []
for date in ew_norebal['date'].unique():
    if date == startDate:
        tradevalue = initcashbal - initEq
    else:
        tradevalue = 0
    
    cash_rows.append({
        'ticker': 'Cash',
        'date': date,
        'close': 0,
        'shares': 0,
        'exposure': initcashbal,
        'trades': 0,
        'tradevalue': tradevalue,
        'commission': 0
    })

cash_df = pd.DataFrame(cash_rows)
ew_norebal = pd.concat([ew_norebal, cash_df], ignore_index=True)
ew_norebal = ew_norebal.sort_values('date').reset_index(drop=True)

print(f"Buy and hold backtest created with {len(ew_norebal)} rows")
ew_norebal.head(10)

## Visualizations

In [ ]:
# Stacked area chart
pivot_data = ew_norebal.pivot(index='date', columns='ticker', values='exposure')
plt.figure(figsize=(12, 6))
plt.stackplot(pivot_data.index, pivot_data.T, labels=pivot_data.columns, alpha=0.8)
plt.xlabel('Date')
plt.ylabel('Exposure ($)')
plt.title('3 ETF USD Risk Premia - Equal Weight, No Rebalancing')
plt.legend(loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Trades chart
trades_data = ew_norebal[ew_norebal['ticker'] != 'Cash']
fig, ax = plt.subplots(figsize=(12, 6))
tickers = sorted(trades_data['ticker'].unique())
width = 20  # bar width in days
for i, ticker in enumerate(tickers):
    ticker_trades = trades_data[trades_data['ticker'] == ticker]
    offset = (i - len(tickers)/2) * width
    ax.bar(ticker_trades['date'] + pd.Timedelta(days=offset), 
           ticker_trades['tradevalue'], 
           width=width, 
           label=ticker)
ax.set_xlabel('Date')
ax.set_ylabel('Trade Value ($)')
ax.set_title('3 ETF USD Risk Premia - Trades')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Commission chart
plt.figure(figsize=(12, 6))
pivot_comm = ew_norebal.pivot(index='date', columns='ticker', values='commission')
pivot_comm.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.xlabel('Date')
plt.ylabel('Commission ($)')
plt.title('3 ETF USD Risk Premia - Commission ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Commission as percentage of exposure
comm_pct = ew_norebal.copy()
comm_pct['commissionpct'] = comm_pct['commission'] / comm_pct['exposure'].replace(0, np.nan)
comm_pct = comm_pct[comm_pct['ticker'] != 'Cash']

fig, ax = plt.subplots(figsize=(12, 6))
for i, ticker in enumerate(tickers):
    ticker_data = comm_pct[comm_pct['ticker'] == ticker]
    offset = (i - len(tickers)/2) * width
    ax.bar(ticker_data['date'] + pd.Timedelta(days=offset), 
           ticker_data['commissionpct'], 
           width=width, 
           label=ticker)
ax.set_xlabel('Date')
ax.set_ylabel('Commission as % of Exposure')
ax.set_title('3 ETF USD Risk Premia - Commission as pct of exposure')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Performance Metrics

In [ ]:
# Calculate portfolio returns
norebal_portfolioreturn = ew_norebal.groupby('date').agg({
    'exposure': 'sum',
    'commission': 'sum'
}).reset_index()
norebal_portfolioreturn.columns = ['date', 'totalequity', 'totalcommission']
norebal_portfolioreturn = norebal_portfolioreturn.sort_values('date')
norebal_portfolioreturn['returns'] = norebal_portfolioreturn['totalequity'].pct_change()

# Performance metrics
returns = norebal_portfolioreturn['returns'].dropna()
annual_return = (1 + returns.mean())**12 - 1
annual_vol = returns.std() * np.sqrt(12)
sharpe = annual_return / annual_vol

# Maximum drawdown
cumulative = (1 + returns).cumprod()
running_max = cumulative.expanding().max()
drawdown = (cumulative - running_max) / running_max
max_drawdown = drawdown.min()

print("Summary Performance Metrics:")
print(f"Annualized Return: {annual_return:.2%}")
print(f"Annualized Volatility: {annual_vol:.2%}")
print(f"Sharpe Ratio: {sharpe:.2f}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")
print(f"\nTotal Commission: ${norebal_portfolioreturn['totalcommission'].sum():.2f}")
print(f"Turnover: 0 (buy and hold)")

totalprofit = norebal_portfolioreturn['totalequity'].iloc[-1] - initEq
costprofit = norebal_portfolioreturn['totalcommission'].sum() / totalprofit
print(f"Total Profit: ${totalprofit:.2f}")
print(f"Trading costs as % of profit: {costprofit:.2%}")

# Tab 2: Equal Dollar Weight with Rebalancing

Inputs for this strategy:

In [ ]:
initEq = 10000  # Initial equity
perShareComm = 0.005  # Per-share commission
minCommPerOrder = 1  # Minimum commission per order
rebalFrequency = 1  # Rebalance frequency in months (1 to 12)
capFrequency = 1  # Frequency to capitalise profits (0 to 12, 0 = don't capitalise)

In [ ]:
assert rebalFrequency > 0, "rebalFrequency must be > 0"

# Create wide dataframes for vectorized backtest
wideprices = monthlyprices.pivot(index='date', columns='ticker', values='close')

# Initialize tracking variables
rowlist = []
cash = initEq
sharepos = np.array([0., 0., 0.])
equity = initEq
capEquity = initEq  # Sticky equity for capitalisation frequency

# Iterate through prices
for i in range(len(wideprices)):
    currentdate = wideprices.index[i]
    currentprice = wideprices.iloc[i].values
    equity = np.sum(sharepos * currentprice) + cash
    
    # Update capEquity if it's re-capitalisation time
    if capFrequency > 0 and i % capFrequency == 0:
        capEquity = equity
    
    # Update position sizing if it's rebalance time
    if i == 0 or i % rebalFrequency == 0:
        targetshares = np.floor((capEquity / 3) / currentprice)
    
    trades = targetshares - sharepos
    tradevalue = trades * currentprice
    commissions = np.abs(trades) * perShareComm
    commissions = np.where(commissions < minCommPerOrder, minCommPerOrder, commissions)
    
    # Adjust cash
    cash = cash - np.sum(tradevalue) - np.sum(commissions)
    sharepos = targetshares.copy()
    sharevalue = sharepos * currentprice
    equity = np.sum(sharevalue) + cash
    
    # Create dataframe row
    tickers = wideprices.columns.tolist()
    row_df = pd.DataFrame({
        'ticker': ['cash'] + tickers,
        'date': [currentdate] * 4,
        'close': [0] + currentprice.tolist(),
        'shares': [0] + sharepos.tolist(),
        'exposure': [cash] + sharevalue.tolist(),
        'sharetrades': [0] + trades.tolist(),
        'tradevalue': [-np.sum(tradevalue)] + tradevalue.tolist(),
        'commission': [0] + commissions.tolist()
    })
    
    rowlist.append(row_df)

# Combine into single dataframe
ew_rebal = pd.concat(rowlist, ignore_index=True)

print(f"Equal-weight rebalancing backtest created with {len(ew_rebal)} rows")
ew_rebal.head(10)

## Visualizations

In [ ]:
# Stacked area chart
pivot_data = ew_rebal.pivot(index='date', columns='ticker', values='exposure')
plt.figure(figsize=(12, 6))
plt.stackplot(pivot_data.index, pivot_data.T, labels=pivot_data.columns, alpha=0.8)
plt.xlabel('Date')
plt.ylabel('Exposure ($)')
plt.title('3 ETF USD Risk Premia - Equal Weight, Rebalancing')
plt.legend(loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Commission chart
plt.figure(figsize=(12, 6))
pivot_comm = ew_rebal.pivot(index='date', columns='ticker', values='commission')
pivot_comm.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.xlabel('Date')
plt.ylabel('Commission ($)')
plt.title('3 ETF USD Risk Premia - Commission ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Trades as % of position size
trades_pct = ew_rebal[ew_rebal['ticker'] != 'cash'].copy()
trades_pct['tradepct'] = trades_pct['tradevalue'] / trades_pct['exposure'].replace(0, np.nan)

fig, ax = plt.subplots(figsize=(12, 6))
tickers = sorted(trades_pct['ticker'].unique())
width = 20
for i, ticker in enumerate(tickers):
    ticker_data = trades_pct[trades_pct['ticker'] == ticker]
    offset = (i - len(tickers)/2) * width
    ax.bar(ticker_data['date'] + pd.Timedelta(days=offset), 
           ticker_data['tradepct'], 
           width=width, 
           label=ticker)
ax.set_xlabel('Date')
ax.set_ylabel('Trade as % of Position')
ax.set_title('3 ETF USD Risk Premia - Trades (as % of position size)')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Performance Metrics

In [ ]:
# Calculate portfolio returns
rebal_portfolioreturn = ew_rebal.groupby('date').agg({
    'exposure': 'sum',
    'commission': 'sum'
}).reset_index()
rebal_portfolioreturn.columns = ['date', 'totalequity', 'totalcommission']
rebal_portfolioreturn = rebal_portfolioreturn.sort_values('date')
rebal_portfolioreturn['returns'] = rebal_portfolioreturn['totalequity'].pct_change()

# Performance metrics
returns = rebal_portfolioreturn['returns'].dropna()
annual_return = (1 + returns.mean())**12 - 1
annual_vol = returns.std() * np.sqrt(12)
sharpe = annual_return / annual_vol

# Maximum drawdown
cumulative = (1 + returns).cumprod()
running_max = cumulative.expanding().max()
drawdown = (cumulative - running_max) / running_max
max_drawdown = drawdown.min()

print("Summary Performance Metrics:")
print(f"Annualized Return: {annual_return:.2%}")
print(f"Annualized Volatility: {annual_vol:.2%}")
print(f"Sharpe Ratio: {sharpe:.2f}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")
print(f"\nTotal Commission: ${rebal_portfolioreturn['totalcommission'].sum():.2f}")

# Calculate turnover
totalselltrades = ew_rebal[
    (ew_rebal['ticker'] != 'cash') & (ew_rebal['tradevalue'] < 0)
]['tradevalue'].sum()
meanequity = rebal_portfolioreturn['totalequity'].mean()
num_years = (endDate.year - startDate.year)
turnover = -totalselltrades / (meanequity * num_years) if num_years > 0 else 0
print(f"Annual Turnover: {turnover:.2f}")

totalprofit = rebal_portfolioreturn['totalequity'].iloc[-1] - initEq
costprofit = rebal_portfolioreturn['totalcommission'].sum() / totalprofit
print(f"Total Profit: ${totalprofit:.2f}")
print(f"Trading costs as % of profit: {costprofit:.2%}")

# Tab 3: Simple Risk Parity

Inputs for this strategy:

In [ ]:
initEq = 10000  # Initial equity
perShareComm = 0.005  # Per-share commission
minCommPerOrder = 1  # Minimum commission per order
rebalFrequency = 1  # Rebalance frequency in months
capFrequency = 1  # Frequency to capitalise profits
assetVolTarget = 0.05  # Asset volatility target (0.01 to 0.1)
volLookback = 30  # Volatility lookback period in days (5 to 250)

## Calculate Volatility Sizing

In [ ]:
# Calculate daily volatility target sizing
theosize_daily = prices.copy()
theosize_daily = theosize_daily.sort_values(['ticker', 'date'])

# Calculate returns
theosize_daily['returns'] = theosize_daily.groupby('ticker')['closeadjusted'].pct_change()

# Calculate rolling volatility (annualized)
theosize_daily['vol'] = theosize_daily.groupby('ticker')['returns'].transform(
    lambda x: x.rolling(window=volLookback, min_periods=volLookback).std() * np.sqrt(252)
)

# Calculate theoretical size (lagged)
theosize_daily['theosize'] = (assetVolTarget / theosize_daily['vol']).shift(1)

# Calculate total size and adjustment factor to constrain leverage to 1
totalsize = theosize_daily.groupby('date')['theosize'].sum().reset_index()
totalsize.columns = ['date', 'totalsize']
totalsize['adjfactor'] = np.where(totalsize['totalsize'] > 1, 1 / totalsize['totalsize'], 1)

# Apply constraint
theosize_constrained = theosize_daily.merge(totalsize, on='date')
theosize_constrained['theosize_constrained'] = (
    theosize_constrained['theosize'] * theosize_constrained['adjfactor']
)
theosize_constrained = theosize_constrained[[
    'ticker', 'date', 'closeadjusted', 'returns', 'vol', 'theosize', 'theosize_constrained'
]].dropna()

# Get month-end snapshots
volsizeprices = monthlyprices.merge(
    theosize_constrained[['ticker', 'date', 'theosize_constrained']], 
    on=['ticker', 'date']
)

print(f"Calculated volatility sizing for {len(volsizeprices)} rows")
volsizeprices.head()

## Plot Theoretical Constrained Sizing

In [ ]:
# Stacked area chart of theoretical sizing
pivot_sizing = volsizeprices.pivot(index='date', columns='ticker', values='theosize_constrained')
plt.figure(figsize=(12, 6))
plt.stackplot(pivot_sizing.index, pivot_sizing.T, labels=pivot_sizing.columns, alpha=0.8)
plt.xlabel('Date')
plt.ylabel('Sizing (% of Portfolio)')
plt.title('3 ETF USD Risk Premia - Theoretical Constrained Sizing')
plt.legend(loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Backtest Volatility Targeting Strategy

In [ ]:
assert rebalFrequency > 0, "rebalFrequency must be > 0"

# Create wide dataframes
wideprices = volsizeprices.pivot(index='date', columns='ticker', values='close')
widetheosize = volsizeprices.pivot(index='date', columns='ticker', values='theosize_constrained')

# Initialize tracking variables
rowlist = []
cash = initEq
sharepos = np.array([0., 0., 0.])
equity = initEq
capEquity = initEq

# Iterate through prices
for i in range(len(wideprices)):
    currentdate = wideprices.index[i]
    currentprice = wideprices.iloc[i].values
    currenttheosize = widetheosize.iloc[i].values
    equity = np.sum(sharepos * currentprice) + cash
    
    # Update capEquity if it's re-capitalisation time
    if capFrequency > 0 and i % capFrequency == 0:
        capEquity = equity
    
    # Update position sizing if it's rebalance time
    if i == 0 or i % rebalFrequency == 0:
        targetshares = np.floor((capEquity * currenttheosize) / currentprice)
    
    trades = targetshares - sharepos
    tradevalue = trades * currentprice
    commissions = np.abs(trades) * perShareComm
    commissions = np.where(commissions < minCommPerOrder, minCommPerOrder, commissions)
    
    # Adjust cash
    cash = cash - np.sum(tradevalue) - np.sum(commissions)
    sharepos = targetshares.copy()
    sharevalue = sharepos * currentprice
    equity = np.sum(sharevalue) + cash
    
    # Create dataframe row
    tickers = wideprices.columns.tolist()
    row_df = pd.DataFrame({
        'ticker': ['cash'] + tickers,
        'date': [currentdate] * 4,
        'close': [0] + currentprice.tolist(),
        'shares': [0] + sharepos.tolist(),
        'exposure': [cash] + sharevalue.tolist(),
        'sharetrades': [0] + trades.tolist(),
        'tradevalue': [-np.sum(tradevalue)] + tradevalue.tolist(),
        'commission': [0] + commissions.tolist()
    })
    
    rowlist.append(row_df)

# Combine into single dataframe
volsize_rebal = pd.concat(rowlist, ignore_index=True)

print(f"Risk parity backtest created with {len(volsize_rebal)} rows")
volsize_rebal.head(10)

## Visualizations

In [ ]:
# Stacked area chart
pivot_data = volsize_rebal.pivot(index='date', columns='ticker', values='exposure')
plt.figure(figsize=(12, 6))
plt.stackplot(pivot_data.index, pivot_data.T, labels=pivot_data.columns, alpha=0.8)
plt.xlabel('Date')
plt.ylabel('Exposure ($)')
plt.title('3 ETF USD Risk Premia - Simple Risk Parity')
plt.legend(loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Commission chart
plt.figure(figsize=(12, 6))
pivot_comm = volsize_rebal.pivot(index='date', columns='ticker', values='commission')
pivot_comm.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.xlabel('Date')
plt.ylabel('Commission ($)')
plt.title('3 ETF USD Risk Premia - Commission ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Trades as % of position size
trades_pct = volsize_rebal[volsize_rebal['ticker'] != 'cash'].copy()
trades_pct['tradepct'] = trades_pct['tradevalue'] / trades_pct['exposure'].replace(0, np.nan)

fig, ax = plt.subplots(figsize=(12, 6))
tickers = sorted(trades_pct['ticker'].unique())
width = 20
for i, ticker in enumerate(tickers):
    ticker_data = trades_pct[trades_pct['ticker'] == ticker]
    offset = (i - len(tickers)/2) * width
    ax.bar(ticker_data['date'] + pd.Timedelta(days=offset), 
           ticker_data['tradepct'], 
           width=width, 
           label=ticker)
ax.set_xlabel('Date')
ax.set_ylabel('Trade as % of Position')
ax.set_title('3 ETF USD Risk Premia - Trades (as % of position size)')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Performance Metrics

In [ ]:
# Calculate portfolio returns
volsize_portfolioreturn = volsize_rebal.groupby('date').agg({
    'exposure': 'sum',
    'commission': 'sum'
}).reset_index()
volsize_portfolioreturn.columns = ['date', 'totalequity', 'totalcommission']
volsize_portfolioreturn = volsize_portfolioreturn.sort_values('date')
volsize_portfolioreturn['returns'] = volsize_portfolioreturn['totalequity'].pct_change()

# Performance metrics
returns = volsize_portfolioreturn['returns'].dropna()
annual_return = (1 + returns.mean())**12 - 1
annual_vol = returns.std() * np.sqrt(12)
sharpe = annual_return / annual_vol

# Maximum drawdown
cumulative = (1 + returns).cumprod()
running_max = cumulative.expanding().max()
drawdown = (cumulative - running_max) / running_max
max_drawdown = drawdown.min()

print("Summary Performance Metrics:")
print(f"Annualized Return: {annual_return:.2%}")
print(f"Annualized Volatility: {annual_vol:.2%}")
print(f"Sharpe Ratio: {sharpe:.2f}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")
print(f"\nTotal Commission: ${volsize_portfolioreturn['totalcommission'].sum():.2f}")

# Calculate turnover
totalselltrades = volsize_rebal[
    (volsize_rebal['ticker'] != 'cash') & (volsize_rebal['tradevalue'] < 0)
]['tradevalue'].sum()
meanequity = volsize_portfolioreturn['totalequity'].mean()
num_years = (endDate.year - startDate.year)
turnover = -totalselltrades / (meanequity * num_years) if num_years > 0 else 0
print(f"Annual Turnover: {turnover:.2f}")

totalprofit = volsize_portfolioreturn['totalequity'].iloc[-1] - initEq
costprofit = volsize_portfolioreturn['totalcommission'].sum() / totalprofit
print(f"Total Profit: ${totalprofit:.2f}")
print(f"Trading costs as % of profit: {costprofit:.2%}")

## Scale to 10% Volatility Target

Scale the portfolio returns to achieve a 10% annualized volatility based on realised volatility

In [ ]:
# Calculate backtested portfolio NAV and returns
port = volsize_rebal.groupby('date')['exposure'].sum().reset_index()
port.columns = ['date', 'nav']
port = port.sort_values('date')
port['returns'] = port['nav'].pct_change()

# Calculate realised volatility and scaling factor
realised_vol = port['returns'].std() * np.sqrt(12)  # Monthly data
vtarget = 0.1  # 10% target volatility
scaling_factor = vtarget / realised_vol

# Scale returns
port['scaled_returns'] = port['returns'] * scaling_factor

# Verify scaled volatility
scaled_vol = port['scaled_returns'].std() * np.sqrt(12)
print(f"Original volatility: {realised_vol:.2%}")
print(f"Scaling factor: {scaling_factor:.2f}")
print(f"Scaled volatility: {scaled_vol:.2%}")
print(f"Target volatility: {vtarget:.2%}")

port.head()

## Write Returns to Disk

Save the scaled portfolio returns for use in the app

In [ ]:
# Save to pickle file
output_data = port[['date', 'scaled_returns']].dropna()

# Create output directory if it doesn't exist
output_dir = Path('..') / 'portfolio' / 'data'
output_dir.mkdir(parents=True, exist_ok=True)

output_path = output_dir / 'rp_returns_volten.pkl'
with open(output_path, 'wb') as f:
    pickle.dump(output_data, f)

print(f"Saved scaled portfolio returns to {output_path}")
print(f"\nSaved {len(output_data)} rows of returns")
output_data.head()